In [2]:
import pandas as pd
import numpy as np

masterdf = pd.read_csv('data/masterdf_20170920.csv', encoding='ISO-8859-1', low_memory=False, 
                      usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23])

# usecols=[1, 2, 3, 6,7,], 

In [3]:

    
masterdf.rename(columns={'count all complaints':'num_complaints'}, inplace=True)

# let's get the columns and their indices
for idx, col in enumerate(masterdf.columns):
    print(idx, col)

#masterdf.head(3)

0 Incident Date
1 EAS
2 Incident_Year
3 Incident_Cat
4 Incident_Dummy
5 Neighborhood
6 Location_y
7 Address
8 Building_Cat
9 Yr_Property_Built
10 Num_Bathrooms
11 Num_Bedrooms
12 Num_Rooms
13 Num_Stories
14 Num_Units
15 Perc_Ownership
16 Land_Value
17 Property_Area
18 Assessed_Improvement_Val
19 Tot_Rooms
20 landval_psqft
21 num_complaints


In [4]:
print('dimensions of the dataset: ', masterdf.shape)

# oooooh! Incident_Dummy is our label! 
print('number of fire incidents: ', len(masterdf[masterdf.Incident_Dummy==1]))

# each building in SF has an EAS code
print('number of EAS codes (buildings): ', len(masterdf.EAS.unique()))

dimensions of the dataset:  (195308, 22)
number of fire incidents:  24791
number of EAS codes (buildings):  182644


In [8]:
#### SETUP ####
#!pip install censusgeocode
# package from US Census Geocoder API
from censusgeocode import CensusGeocode

cg = CensusGeocode()

# need to format Location_y differently in order to input to censusgeocoder
def split_coords(inputstring): 
    coords = inputstring.strip('()').split(',')
    coords = [float(i) for i in coords]
    return coords

masterdf['latlong'] = masterdf['Location_y'].apply(split_coords)
masterdf = masterdf.drop('Location_y', axis=1)

# function to get the census block from the geo coordinates
def get_block(coords):
    res = cg.coordinates(x=coords[1],y=coords[0])
    block = res[0].get('2010 Census Blocks')[0].get('BLOCK')
    print(block) # so we can see its progress
    return block

# function to split up masterdf.latlong into subsets
def split_seq(seq, size):
    newseq = []
    splitsize = 1.0/size*len(seq)
    for i in range(size):
        newseq.append(seq[int(round(i*splitsize)):int(round((i+1)*splitsize))])
    return newseq

# gonna try splitting it up and running one little chunk at a time and saving it
coords_subsets = split_seq(masterdf.latlong, 100)



In [36]:
#### testing #####
coords = coords_subsets[0][0]
print(coords)
cg.coordinates(x=coords[1], y=coords[0])


[37.7543289339354, -122.480327187833]


[{'2010 Census Blocks': [{'AREALAND': 19651,
    'AREAWATER': 0,
    'BASENAME': '3000',
    'BLKGRP': '3',
    'BLOCK': '3000',
    'CENTLAT': '+37.7550089',
    'CENTLON': '-122.4802883',
    'COUNTY': '075',
    'FUNCSTAT': 'S',
    'GEOID': '060750328023000',
    'INTPTLAT': '+37.7550089',
    'INTPTLON': '-122.4802883',
    'LSADC': 'BK',
    'LWBLKTYP': 'L',
    'MTFCC': 'G5040',
    'NAME': 'Block 3000',
    'OBJECTID': 741050,
    'OID': 210403968135774,
    'STATE': '06',
    'SUFFIX': '',
    'TRACT': '032802'}],
  'Census Tracts': [{'AREALAND': 548900,
    'AREAWATER': 97204,
    'BASENAME': '328.02',
    'CENTLAT': '+37.7524026',
    'CENTLON': '-122.4814851',
    'COUNTY': '075',
    'FUNCSTAT': 'S',
    'GEOID': '06075032802',
    'INTPTLAT': '+37.7530955',
    'INTPTLON': '-122.4812305',
    'LSADC': 'CT',
    'MTFCC': 'G5020',
    'NAME': 'Census Tract 328.02',
    'OBJECTID': 14110,
    'OID': 207903717106334,
    'STATE': '06',
    'TRACT': '032802'}],
  'Counties': [

In [9]:
print(len(coords_subsets))

%store -r block_subset
print(len(block_subset))
masterdf.head()


100
195308


,Incident Date,EAS,Incident_Year,Incident_Cat,Incident_Dummy,Neighborhood,Address,Building_Cat,Yr_Property_Built,Num_Bathrooms,...,Num_Units,Perc_Ownership,Land_Value,Property_Area,Assessed_Improvement_Val,Tot_Rooms,landval_psqft,num_complaints,census_block,latlong
0,2015-06-20,451005.0,2015.0,COOKING FIRE,1.0,SUNSET/PARKSIDE,1532 NORIEGA ST,COMMERCIAL USE,1989.0,0.0,...,2.777778,1.0,4.384343e+05,4135.0,262181.666667,11.0,106.030069,0.0,3000,"[37.7543289339354, -122.480327187833]"
1,2010-11-28,360149.0,2010.0,COOKING FIRE,1.0,MISSION,135 CAPP ST,APARTMENT,1908.0,0.0,...,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,4000,"[37.7645472195468, -122.418358468789]"
2,2011-04-26,360149.0,2011.0,COOKING FIRE,1.0,MISSION,135 CAPP ST,APARTMENT,1908.0,0.0,...,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,4000,"[37.7645472195468, -122.418358468789]"
3,2006-03-09,360149.0,2006.0,BUILDING FIRE,1.0,MISSION,135 CAPP ST,APARTMENT,1908.0,0.0,...,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,4000,"[37.7645472195468, -122.418358468789]"
4,2004-05-28,360149.0,2004.0,OUTDOOR FIRE,1.0,MISSION,135 CAPP ST,APARTMENT,1908.0,0.0,...,12.000000,1.0,1.365665e+06,9318.0,566375.428571,36.0,146.562030,3.0,4000,"[37.7645472195468, -122.418358468789]"


In [10]:
# to get rid of duplicate entries (overlapping indices)
block_subset = block_subset.groupby(block_subset.index).first()


masterdf['census_block'] = block_subset
print(len(masterdf))

# ahh.. solution to the unhashable list errors!
masterdf['latlong'] = masterdf['latlong'].apply(tuple)

masterdf.columns
#print(len(coords_subsets))

195308


Index(['Incident Date', 'EAS', 'Incident_Year', 'Incident_Cat',
       'Incident_Dummy', 'Neighborhood', 'Address', 'Building_Cat',
       'Yr_Property_Built', 'Num_Bathrooms', 'Num_Bedrooms', 'Num_Rooms',
       'Num_Stories', 'Num_Units', 'Perc_Ownership', 'Land_Value',
       'Property_Area', 'Assessed_Improvement_Val', 'Tot_Rooms',
       'landval_psqft', 'num_complaints', 'census_block', 'latlong'],
      dtype='object')

In [11]:
# need to also merge in the census tracts from another csv file (oops should've used that originally!)
from column_report import get_column_report
import os
import sys
# to import my column_report package
mod_path = os.path.abspath(os.path.join('..'))
if mod_path not in sys.path:
    sys.path.append(mod_path)
    


master_tracts = pd.read_csv('data/masterdf_inc_census_tract.csv', encoding='ISO-8859-1',low_memory=False,
                           usecols=[3,34])

for idx, col in enumerate(master_tracts.columns):
    print(idx, col)

# to ensure we can merge without duplicates    
master_tracts = master_tracts.drop_duplicates()
masterdf = masterdf.drop_duplicates()


master_tracts = master_tracts[master_tracts.EAS.isin(masterdf.EAS)]

masterdf = masterdf.merge(master_tracts, on=['EAS'], how='inner')

print(len(masterdf))
   


0 EAS
1 tract
195184


In [12]:
print('number of unique combinations of (census_block, tract) in the fire incident data: %s' 
      %len(masterdf.groupby(['census_block', 'tract']).size()))


# count the number of buildings in each block
#print(masterdf[['EAS', 'census_block', 'tract']].groupby(['census_block', 'tract']).agg(['count']).reset_index())


number of unique combinations of (census_block, tract) in the fire incident data: 4981


In [94]:
from column_report import get_column_report

#%store -r masterdf
print(masterdf.columns)

get_column_report(masterdf, numerical='int')


Index(['Incident Date', 'EAS', 'Incident_Year', 'Incident_Cat',
       'Incident_Dummy', 'Neighborhood', 'Address', 'Building_Cat',
       'Yr_Property_Built', 'Num_Bathrooms', 'Num_Bedrooms', 'Num_Rooms',
       'Num_Stories', 'Num_Units', 'Perc_Ownership', 'Land_Value',
       'Property_Area', 'Assessed_Improvement_Val', 'Tot_Rooms',
       'landval_psqft', 'num_complaints', 'latlong', 'census_block', 'tract'],
      dtype='object')


,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
0,Incident Date,object,5205,2015-06-20,0,0.00
21,latlong,object,144203,"(37.7543289339354, -122.480327187833)",0,0.00
19,landval_psqft,float64,137246,106.03,0,0.00
16,Property_Area,float64,18303,4135,0,0.00
15,Land_Value,float64,80330,438434,0,0.00
12,Num_Stories,float64,343,3,0,0.00
22,census_block,int64,287,3000,0,0.00
8,Yr_Property_Built,float64,529,1989,0,0.00
23,tract,int64,192,32802,0,0.00
6,Address,object,182603,1532 NORIEGA ST,0,0.00


In [29]:
# had to keep re-running the cg.coordinates() so this cell 
# gets a list (of the indicies) of where census_block is still null
stillneed = pd.Series(masterdf[masterdf['census_block'].isnull()].index)
print(len(stillneed))
# then filter through coords_subsets and remove those we've already ran

# create a separate list with completed subsets removed
coords_stillneed = []


for i, coords in enumerate(coords_subsets):
    # create a series of the indices within current subset
    print(i)
    c = pd.Series(coords.index)
    print(c.isin(stillneed).all())
    # check if we still need the block for these coordinates
    # then we can get rid of it
    # so, if the whole set is contained in the stillneed series
    # (can't use .any() because stillneed also includes blocks we've checked
    # but gave null, so every coordssubset will return true)
    if c.isin(stillneed).all():
        coords_stillneed.append(coords)
        #print(coords_stillneed[i][:3])
        print('keeping subset ', i)



58905
0
False
1
False
2
False
3
False
4
False
5
False
6
False
7
False
8
False
9
False
10
False
11
False
12
False
13
False
14
False
15
False
16
False
17
False
18
False
19
False
20
False
21
False
22
False
23
False
24
False
25
False
26
False
27
False
28
False
29
False
30
False
31
False
32
False
33
False
34
False
35
False
36
False
37
False
38
False
39
False
40
False
41
False
42
False
43
False
44
False
45
False
46
False
47
False
48
False
49
False
50
False
51
False
52
False
53
False
54
False
55
False
56
False
57
False
58
False
59
False
60
False
61
False
62
False
63
False
64
False
65
False
66
False
67
False
68
False
69
False
70
True
keeping subset  70
71
True
keeping subset  71
72
True
keeping subset  72
73
True
keeping subset  73
74
True
keeping subset  74
75
True
keeping subset  75
76
True
keeping subset  76
77
True
keeping subset  77
78
True
keeping subset  78
79
True
keeping subset  79
80
True
keeping subset  80
81
True
keeping subset  81
82
True
keeping subset  82
83
True
keeping subset 

In [32]:

 
# loops through and finds the census block for each row
# divided the whole dataset into 100 subsets, computing blocks for each subset and then storing them
for i in range(len(coords_stillneed)):
    lilblocks = coords_stillneed[i].apply(get_block)
    block_subset = block_subset.append(lilblocks)
    %store block_subset
    print(len(block_subset), ' blocks')
    print('-'*200,'\n',len(coords_stillneed) - i,' bottles of beer on the wall')

1010
1010
1010
1010
1010
1010
1010
1010
1010
1010
1010
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4001
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
4000
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007


In [13]:
masterdf.census_block = pd.to_numeric(masterdf.census_block)

# to replace missing values with the block in the following row (they're usually grouped together)
masterdf.census_block = masterdf.census_block.fillna(0).replace(0,method='bfill').astype(int)

masterdf.census_block.head() #= masterdf.census_block.apply(lambda x: int(x))

# # these cols have mostly zeros and prob won't be useful
# masterdf = masterdf.drop(['Num_Bedrooms', 'num_complaints'], axis=1)

len(masterdf[masterdf.census_block==0])


0

In [95]:
%store masterdf

Stored 'masterdf' (DataFrame)
